###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# American Community Survey 2019 1 Year Estimates

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4702180', #Ashland City: Cheatham
          '1600000US4739660', #Kingston Springs: Cheatham
          '1600000US4757480', #Pegram: Cheatham
          '1600000US4759560', #Pleasant View: Cheatham
          '1600000US4704620', #Belle Meade: Davidson
          '1600000US4705140', #Berry Hill: Davidson
          '1600000US4727020', #Forest Hills: Davidson
          '1600000US4729920', #Goodlettsville: Davidson/Sumner
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4754780', #Oak Hill: Davidson
          '1600000US4763140', #Ridgetop: Davidson/Robertson
          '1600000US4709880', #Burns: Dickson
          '1600000US4713080', #Charlotte: Dickson
          '1600000US4720620', #Dickson: Dickson
          '1600000US4769080', #Slayden: Dickson
          '1600000US4776860', #Vanleer: Dickson
          '1600000US4779980', #White Bluff: Dickson
          '1600000US4724320', #Erin: Houston
          '1600000US4773460', #Tennessee Ridge: Houston/Stewart
          '1600000US4744840', #McEwen: Humphreys
          '1600000US4752820', #New Johnsonville: Humphreys
          '1600000US4778560', #Waverly: Humphreys
          '1600000US4716540', #Columbia: Maury
          '1600000US4751080', #Mount Pleasant: Maury
          '1600000US4770580', #Spring Hill: Maury/Williamson
          '1600000US4715160', #Clarksville: Montgomery
          '1600000US4700200', #Adams: Robertson
          '1600000US4711980', #Cedar Hill: Robertson
          '1600000US4716980', #Coopertown: Robertson
          '1600000US4718420', #Cross Plains: Robertson
          '1600000US4730960', #Greenbrier: Robertson
          '1600000US4748980', #Millersville: Robertson/Sumner
          '1600000US4760280', #Portland: Robertson/Sumner
          '1600000US4770500', #Springfield: Robertson
          '1600000US4780200', #White House: Robertson/Sumner
          '1600000US4722360', #Eagleville: Rutherford
          '1600000US4741200', #La Vergne: Rutherford
          '1600000US4751560', #Murfreesboro: Rutherford
          '1600000US4769420', #Smyrna: Rutherford
          '1600000US4718820', #Cumberland City: Stewart
          '1600000US4721400', #Dover: Stewart
          '1600000US4728540', #Gallatin: Sumner
          '1600000US4733280', #Hendersonville: Sumner
          '1600000US4779420', #Westmoreland: Sumner
          '1600000US4708280', #Brentwood: Williamson
          '1600000US4725440', #Fairview: Williamson
          '1600000US4727740', #Franklin: Williamson
          '1600000US4753460', #Nolensville: Williamson
          '1600000US4773900', #Thompson's Station: Williamson
          '1600000US4741520', #Lebanon: Wilson
          '1600000US4750780', #Mount Juliet: Wilson
          '1600000US4778320', #Watertown: Wilson
          '1600000US4740160', #Lafayette: Macon
          '1600000US2169114', #Scottsville city, Kentucky
          '1600000US2128918', #Franklin city, Kentucky
          '1600000US4749460'] #Mitchelville city: Sumner

In [5]:
def percent(x, y):
    return (x/y)*100

## Read In Data Guide

In [6]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS1YR.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [7]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
#dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [8]:
url_str= 'https://api.census.gov/data/2019/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'

In [9]:
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [10]:
data = savename
data = data.set_index(['NAME', 'GEO_ID'])
data.head(2)

,,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS
NAME,GEO_ID,,,,,,,,
"Robertson County, Tennessee",0500000US47147,None,None,None,None,None,None,47,147
"Williamson County, Tennessee",0500000US47187,236055,206143,10780,7929,10386,817,47,187


In [11]:
transp = data.transpose()
transp.head(10)

NAME,"Robertson County, Tennessee","Williamson County, Tennessee","Rutherford County, Tennessee","Wilson County, Tennessee","Sumner County, Tennessee","Davidson County, Tennessee","Montgomery County, Tennessee","Maury County, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","Franklin city, Tennessee","Murfreesboro city, Tennessee","Clarksville city, Tennessee",Tennessee,United States
GEO_ID,0500000US47147,0500000US47187,0500000US47149,0500000US47189,0500000US47165,0500000US47037,0500000US47125,0500000US47119,1600000US4752006,1600000US4727740,1600000US4751560,1600000US4715160,0400000US47,0100000US
geo_total_series,None,236055,328286,142941,189646,682750,205743,None,657228,None,None,155355,6754461,324665523
geo_samehouse_total,None,206143,270083,123474,165036,548768,169104,None,526710,None,None,122838,5782145,280343146
geo_movedsamecounty_total,None,10780,30636,9071,10544,76321,15666,None,74793,None,None,12792,530719,24512131
geo_moveddifferentcountysamestate_total,None,7929,18078,5448,8283,22145,5579,None,20948,None,None,4991,224464,10501717
geo_moveddifferentstate_total,None,10386,8890,4848,5374,28381,14258,None,27642,None,None,13680,196765,7398337
geo_movedabroad_total,None,817,599,100,409,7135,1136,None,7135,None,None,1054,20368,1910192
StateFIPS,47,47,47,47,47,47,47,47,47,47,47,47,47,1
GeoFIPS,147,187,149,189,165,037,125,119,52006,27740,51560,15160,0,0


In [12]:
transp = transp.drop(columns = [('Maury County, Tennessee', '0500000US47119'), ('Robertson County, Tennessee', '0500000US47147'),
                                ('Murfreesboro city, Tennessee', '1600000US4751560'), ('Franklin city, Tennessee', '1600000US4727740')])

In [13]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [14]:
data = transp

In [15]:
data.head()

NAME,"Williamson County, Tennessee","Rutherford County, Tennessee","Wilson County, Tennessee","Sumner County, Tennessee","Davidson County, Tennessee","Montgomery County, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","Clarksville city, Tennessee",Tennessee,United States
GEO_ID,0500000US47187,0500000US47149,0500000US47189,0500000US47165,0500000US47037,0500000US47125,1600000US4752006,1600000US4715160,0400000US47,0100000US
geo_total_series,236055.0,328286.0,142941.0,189646.0,682750.0,205743.0,657228.0,155355.0,6754461.0,324665523.0
geo_samehouse_total,206143.0,270083.0,123474.0,165036.0,548768.0,169104.0,526710.0,122838.0,5782145.0,280343146.0
geo_movedsamecounty_total,10780.0,30636.0,9071.0,10544.0,76321.0,15666.0,74793.0,12792.0,530719.0,24512131.0
geo_moveddifferentcountysamestate_total,7929.0,18078.0,5448.0,8283.0,22145.0,5579.0,20948.0,4991.0,224464.0,10501717.0
geo_moveddifferentstate_total,10386.0,8890.0,4848.0,5374.0,28381.0,14258.0,27642.0,13680.0,196765.0,7398337.0


### Not aggregating for regions or unincorporated areas as data is only available for select geographies.

In [16]:
data = data.transpose()
coldropper = data
data = data.reset_index()

In [17]:
#calculations
#geographic mobility
data['GeoMobility:Total'] = data['geo_total_series']
data['GeoMobility:Same House'] = data['geo_samehouse_total']
data['GeoMobility%:Same House'] = percent(data['GeoMobility:Same House'],data['GeoMobility:Total'])
data['GeoMobility:Moved Same County'] = data['geo_movedsamecounty_total']
data['GeoMobility%:Moved Same County'] = percent(data['GeoMobility:Moved Same County'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different County Same State'] = data['geo_moveddifferentcountysamestate_total']
data['GeoMobility%:Moved Different County Same State'] = percent(data['GeoMobility:Moved Different County Same State'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different State'] = data['geo_moveddifferentstate_total']
data['GeoMobility%:Moved Different State'] = percent(data['GeoMobility:Moved Different State'],data['GeoMobility:Total'])
data['GeoMobility:Moved From Abroad'] = data['geo_movedabroad_total']
data['GeoMobility%:Moved From Abroad'] = percent(data['GeoMobility:Moved From Abroad'],data['GeoMobility:Total'])

In [18]:
cols = coldropper.columns
data = data.drop(columns = cols)
data.head(3)

,NAME,GEO_ID,GeoMobility:Total,GeoMobility:Same House,GeoMobility%:Same House,GeoMobility:Moved Same County,GeoMobility%:Moved Same County,GeoMobility:Moved Different County Same State,GeoMobility%:Moved Different County Same State,GeoMobility:Moved Different State,GeoMobility%:Moved Different State,GeoMobility:Moved From Abroad,GeoMobility%:Moved From Abroad
0,"Williamson County, Tennessee",0500000US47187,236055.0,206143.0,87.328377,10780.0,4.566732,7929.0,3.358963,10386.0,4.399822,817.0,0.346106
1,"Rutherford County, Tennessee",0500000US47149,328286.0,270083.0,82.270642,30636.0,9.332107,18078.0,5.506784,8890.0,2.708005,599.0,0.182463
2,"Wilson County, Tennessee",0500000US47189,142941.0,123474.0,86.381094,9071.0,6.345975,5448.0,3.811363,4848.0,3.391609,100.0,0.069959


In [19]:
data.to_csv('../../Outputs/2019ACS1YR.csv', index = False)